In [3]:
from pulp import LpMaximize, LpProblem, LpVariable, PULP_CBC_CMD

# Define the problem
model = LpProblem(name="mixed-integer-programming", sense=LpMaximize)

# Define variables (integer constraints)
A = LpVariable(name="A", lowBound=0, cat="Integer")  # Product A (integer)
B = LpVariable(name="B", lowBound=0, cat="Integer")  # Product B (integer)

# Objective function: Maximize profit
model += 50 * A + 40 * B, "Total_Profit"

# Constraints
model += 2 * A + 1 * B <= 100, "Time_Constraint"  # Available working hours
model += A <= 30, "Raw_Material_A_Constraint"  # Max supply for A
model += B <= 40, "Raw_Material_B_Constraint"  # Max supply for B

# Solve the problem
model.solve(PULP_CBC_CMD(msg=0))

# Print results
print(f"Optimal Production Plan:")
print(f"Product A: {A.varValue} units")
print(f"Product B: {B.varValue} units")
print(f"Maximum Profit: ${model.objective.value()}")

Optimal Production Plan:
Product A: 30.0 units
Product B: 40.0 units
Maximum Profit: $3100.0


In [5]:
from pulp import LpMaximize, LpProblem, LpVariable, PULP_CBC_CMD

# Define the problem
model = LpProblem(name="complex-mip", sense=LpMaximize)

# Decision Variables
X = LpVariable(name="X", lowBound=0, cat="Integer")  # Product X (integer)
Y = LpVariable(name="Y", lowBound=0, cat="Integer")  # Product Y (integer)
Z_X = LpVariable(name="Z_X", cat="Binary")  # Binary variable for X production
Z_Y = LpVariable(name="Z_Y", cat="Binary")  # Binary variable for Y production

# Objective Function: Maximize revenue
model += 50 * X + 40 * Y, "Total_Revenue"

# Constraints
model += X <= 100,     "Max_X_Production"
model += Y <= 100,     "Max_Y_Production"
model += X + Y <= 100, "Total_Production_Limit"
model += X >= 10,      "Min_X_Demand"
model += Y >= 10,      "Min_Y_Demand"

# Budget Constraint
model += (X * 20 + Y * 40) <= 3000, "Budget_Constraint"

# Logical Constraint: If a product is produced, its binary variable must be 1
model += X <= 100 * Z_X, "Binary_X_Constraint"
model += Y <= 100 * Z_Y, "Binary_Y_Constraint"

# Constraint: If X is produced, at least 5 more units of Y must be produced
model += X + 5 <= Y, "X_Less_Than_Y"

# Solve the model
model.solve(PULP_CBC_CMD(msg=0))

# Print results
print(f"Optimal Production Plan:")
print(f"Product X: {X.varValue} units")
print(f"Product Y: {Y.varValue} units")
print(f"Maximum Revenue: ${model.objective.value()}")

Optimal Production Plan:
Product X: 46.0 units
Product Y: 52.0 units
Maximum Revenue: $4380.0


In [6]:
from pulp import LpMaximize, LpMinimize, LpProblem, LpVariable, lpSum, PULP_CBC_CMD

# Define the problem
model = LpProblem(name="logistics-milp", sense=LpMinimize)

# Warehouses and their supply capacity
warehouses = ["W1", "W2"]
supply = {"W1": 500, "W2": 600}

# Stores and their demand
stores = ["S1", "S2", "S3"]
demand = {"S1": 300, "S2": 400, "S3": 300}

# Transportation cost per unit from warehouses to stores
cost = {
    ("W1", "S1"): 4, ("W1", "S2"): 6, ("W1", "S3"): 9,
    ("W2", "S1"): 5, ("W2", "S2"): 4, ("W2", "S3"): 7
}

# Decision Variables: Number of units shipped from each warehouse to each store
shipment = {(w, s): LpVariable(name=f"ship_{w}_{s}", lowBound=0, cat="Integer") for w in warehouses for s in stores}

# Objective Function: Minimize transportation cost
model += lpSum(cost[w, s] * shipment[w, s] for w in warehouses for s in stores), "Total_Transportation_Cost"

# Constraints
# Supply constraints: Each warehouse cannot ship more than its supply
for w in warehouses:
    model += lpSum(shipment[w, s] for s in stores) <= supply[w], f"Supply_Limit_{w}"

# Demand constraints: Each store must receive its required demand
for s in stores:
    model += lpSum(shipment[w, s] for w in warehouses) >= demand[s], f"Demand_Fulfillment_{s}"

# Solve the model
model.solve(PULP_CBC_CMD(msg=0))

# Print results
print("Optimal Shipment Plan:")
for w in warehouses:
    for s in stores:
        print(f"Ship from {w} to {s}: {shipment[w, s].varValue} units")

print(f"Minimum Transportation Cost: ${model.objective.value()}")

Optimal Shipment Plan:
Ship from W1 to S1: 300.0 units
Ship from W1 to S2: 100.0 units
Ship from W1 to S3: 0.0 units
Ship from W2 to S1: 0.0 units
Ship from W2 to S2: 300.0 units
Ship from W2 to S3: 300.0 units
Minimum Transportation Cost: $5100.0


In [13]:
from pulp import LpMinimize, LpProblem, LpVariable, lpSum, PULP_CBC_CMD

# Define the problem
model = LpProblem(name="logistics-milp-advanced", sense=LpMinimize)

# Warehouses and their supply capacity
warehouses = ["W1", "W2"]
supply = {"W1": 500, "W2": 600}

# Stores and their demand
stores = ["S1", "S2", "S3"]
demand = {"S1": 300, "S2": 400, "S3": 300}

# Base transportation cost per unit from warehouse to store
base_cost = {
    ("W1", "S1"): 4, ("W1", "S2"): 6, ("W1", "S3"): 9,
    ("W2", "S1"): 5, ("W2", "S2"): 4, ("W2", "S3"): 7
}

# Capacity constraints per route (Max shipment allowed on each route)
capacity = {
    ("W1", "S1"): 250, ("W1", "S2"): 300, ("W1", "S3"): 200,
    ("W2", "S1"): 200, ("W2", "S2"): 250, ("W2", "S3"): 250
}

# Dynamic cost threshold: Above this amount, additional cost applies
threshold = 100  
extra_cost = 5  # Extra cost per unit above the threshold

# Decision Variables: Number of units shipped from each warehouse to each store
shipment = {(w, s): LpVariable(name=f"ship_{w}_{s}", lowBound=0, cat="Integer") for w in warehouses for s in stores}

# Additional variable to track excess shipment (above the threshold)
excess_shipment = {(w, s): LpVariable(name=f"excess_{w}_{s}", lowBound=0, cat="Integer") for w in warehouses for s in stores}

# Objective Function: Minimize transportation cost with dynamic pricing
model += lpSum(base_cost[w, s] * shipment[w, s] + extra_cost * excess_shipment[w, s] for w in warehouses for s in stores), "Total_Transportation_Cost"

# Constraints

# Supply constraints: Each warehouse cannot ship more than its available supply
for w in warehouses:
    model += lpSum(shipment[w, s] for s in stores) <= supply[w], f"Supply_Limit_{w}"

# Demand constraints: Each store must receive its required demand
for s in stores:
    model += lpSum(shipment[w, s] for w in warehouses) >= demand[s], f"Demand_Fulfillment_{s}"

# Capacity constraints: Shipment cannot exceed the max capacity per route
for w, s in capacity:
    model += shipment[w, s] <= capacity[w, s], f"Capacity_Limit_{w}_{s}"

# Dynamic cost constraints: Track excess shipment above the threshold
for w, s in base_cost:
    model += excess_shipment[w, s] >= shipment[w, s] - threshold, f"Excess_Shipment_{w}_{s}"
    model += excess_shipment[w, s] >= 0, f"Non_Negative_Excess_{w}_{s}"

# Solve the model
model.solve(PULP_CBC_CMD(msg=0))

# Print results
print("Optimal Shipment Plan:")
for w in warehouses:
    for s in stores:
        print(f"Ship from {w} to {s}: {shipment[w, s].varValue} units (Excess: {excess_shipment[w, s].varValue} units)")

print(f"Minimum Transportation Cost: ${model.objective.value()}")

Optimal Shipment Plan:
Ship from W1 to S1: 200.0 units (Excess: 100.0 units)
Ship from W1 to S2: 150.0 units (Excess: 50.0 units)
Ship from W1 to S3: 100.0 units (Excess: 0.0 units)
Ship from W2 to S1: 100.0 units (Excess: 0.0 units)
Ship from W2 to S2: 250.0 units (Excess: 150.0 units)
Ship from W2 to S3: 200.0 units (Excess: 100.0 units)
Minimum Transportation Cost: $7500.0
